In [ ]:
# In this file, the End-of-life vehicle (ELV) sector is modelled, based on transfercoefficients (Tk) 
# for 16 different materials. First, the material flows and the resulting matrices of material concentrations are calculated.
# In a second step, the results are employed for a Statistical Entropy Analysis (SEA).
import math
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame
from numpy import errstate,isneginf,array
import plotly.graph_objects as go

######################################
######################################
#Import of data for outflows and inflows of vehicles [in million]
input_file = pd.read_csv("_Main_scenario_file.csv", sep=';', index_col='Year')
S70_out = input_file.loc[:, ['1 Outflow S70 EV','1 Outflow S70 ICEV']]
S100_out = input_file.loc[:, ['2 Outflow S100 EV', '2 Outflow S100 ICEV']]
S70D_out = input_file.loc[:, ['3 Outflow S70D EV', '3 Outflow S70D ICEV']]
S70D_CE_plus_out = input_file.loc[:, ['4 Outflow S70D-CE+ EV', '4 Outflow S70D-CE+ ICEV']]
S100D_CE_plus_out = input_file.loc[:, ['5 Outflow S100D-CE+ EV', '5 Outflow S100D-CE+ ICEV']]

S70_out.columns = ['EV', 'ICEV']
S100_out.columns = ['EV', 'ICEV']
S70D_out.columns = ['EV', 'ICEV']
S70D_CE_plus_out.columns = ['EV', 'ICEV']
S100D_CE_plus_out.columns = ['EV', 'ICEV']

1  #################################
####################################
outflow_df = S70_out          #1
#outflow_df = S100_out         #2
#outflow_df = S70D_out         #3
#outflow_df = S70D_CE_plus_out #4
#outflow_df = S100D_CE_plus_out #5
####################################
####################################

outflow_df.loc[:, 'Total'] = pd.Series(17.4636, index=outflow_df.index)
outflow_df.loc[:, 'EVrel'] = (outflow_df['EV']/outflow_df['Total']) * 100
outflow_df.loc[:, 'ICEVrel'] = (outflow_df['ICEV']/outflow_df['Total']) * 100

#dataframe with given component composition
generic_df_1percent = pd.read_csv("component_composition.csv", sep=';')
generic_df_1percent = generic_df_1percent.set_index('Flow')

outflow_df_to_plot = outflow_df[['EV', 'ICEV']]
outflow_df_to_plot.plot(title="Outflow of EV and ICEV from stock (in million)")

###########
#Inflow of vehicles from production
inflow_df = input_file

#calculation of inflows as relative fractions of EVs and ICEVs per year
inflows = pd.read_csv("_Main_scenario_file.csv", sep=';', index_col='Year')
S70_in = inflows.loc[:, ('1 Inflow S70 EV', '1 Inflow S70 ICEV')]
S100_in = inflows.loc[:, ('2 Inflow S100 EV', '2 Inflow S100 ICEV')]
S70D_in = inflows.loc[:, ('3 Inflow S70D EV', '3 Inflow S70D ICEV')]
S70D_CE_plus_in = inflows.loc[:, ('4 Inflow S70D-CE+ EV', '4 Inflow S70D-CE+ ICEV')]
S100D_CE_plus_in = inflows.loc[:, ('5 Inflow S100D-CE+ EV', '5 Inflow S100D-CE+ ICEV')]

S70_in.columns = ['EV', 'ICEV']
S100_in.columns = ['EV', 'ICEV']
S70D_in.columns = ['EV', 'ICEV']
S70D_CE_plus_in.columns = ['EV', 'ICEV']
S100D_CE_plus_in.columns = ['EV', 'ICEV']

2  #################################
####################################
inflow_df = S70_in          #1
#inflow_df = S100_in         #2
#inflow_df = S70D_in         #3
#inflow_df = S70D_CE_plus_in #4
#inflow_df = S100D_CE_plus_in #5
####################################
####################################

inflow_df.loc[:, 'Total'] = pd.Series(17.4636, index=inflow_df.index)
inflow_df.loc[:, 'EVrel'] = (inflow_df.loc[:, 'EV']/inflow_df.loc[:, 'Total']) * 100
inflow_df.loc[:, 'ICEVrel'] = (inflow_df.loc[:, 'ICEV']/inflow_df.loc[:, 'Total']) * 100

inflow_df_to_plot = inflow_df[['EV', 'ICEV']]
inflow_df_to_plot.plot(title="Inflow of EV and ICEV to stock (in million)")

#import of transfer-coefficient matrix and define an alternative matrix for CE and CE+ scenarios
Tks = pd.read_csv("Tks.csv", sep=';')
Tks = Tks.set_index('Flow')

Tks2 = pd.read_csv("Tks_improved.csv", sep=';') 
Tks2 = Tks2.set_index('Flow')

#import empty dataframe for imported materials into the system to be filled based on demand for production of EVs/ICEVs
empty_df = pd.read_csv('imported_empty.csv', sep=';', index_col='Flow')

#Define results output dataframe
results_appended_df = pd.read_csv("in.csv", sep=',')
results_appended_df = results_appended_df.set_index('Stage')
#results_appended_df

#Outflows of components
outflow_components = pd.read_csv("_components.csv", sep=';')
outflow_components = outflow_components.set_index('Year')

#slicing dataframe into scenario dataframes
outflow_components_1 = outflow_components.loc[:, ('1.Battery_EV', '1.Other_powertrain_ICEV')]
outflow_components_2 = outflow_components.loc[:, ('2.Battery_EV', '2.Other_powertrain_ICEV')]
outflow_components_3 = outflow_components.loc[:, ('3.Battery_EV', '3.Other_powertrain_ICEV')]
outflow_components_4 = outflow_components.loc[:, ('4.Battery_EV', '4.Other_powertrain_ICEV')]
outflow_components_5 = outflow_components.loc[:, ('5.Battery_EV', '5.Other_powertrain_ICEV')]

#Rename column headers to identical names
outflow_components_1.columns = ['Battery_EV','Other_powertrain_ICEV']
outflow_components_2.columns = ['Battery_EV','Other_powertrain_ICEV']
outflow_components_3.columns = ['Battery_EV','Other_powertrain_ICEV']
outflow_components_4.columns = ['Battery_EV','Other_powertrain_ICEV']
outflow_components_5.columns = ['Battery_EV','Other_powertrain_ICEV']

#Calculate relative values
outflow_components_1.loc[:,'Battery_EV_rel'] = (outflow_components_1.loc[:,('Battery_EV')]/outflow_df.loc[:,('Total')]) * 100
outflow_components_1.loc[:,'Other_powertrain_ICEV_rel'] = (outflow_components_1.loc[:,('Other_powertrain_ICEV')]/outflow_df.loc[:,('Total')]) * 100
outflow_components_1.loc[:,'Total'] = outflow_components_1.loc[:,('Battery_EV_rel')] + outflow_components_1.loc[:,('Other_powertrain_ICEV_rel')]

outflow_components_2.loc[:,'Battery_EV_rel'] = (outflow_components_2.loc[:,('Battery_EV')]/outflow_df.loc[:,('Total')]) * 100
outflow_components_2.loc[:,'Other_powertrain_ICEV_rel'] = (outflow_components_2.loc[:,('Other_powertrain_ICEV')]/outflow_df.loc[:,('Total')]) * 100
outflow_components_2.loc[:,'Total'] = outflow_components_2.loc[:,('Battery_EV_rel')] + outflow_components_2.loc[:,('Other_powertrain_ICEV_rel')]

outflow_components_3.loc[:,'Battery_EV_rel'] = (outflow_components_3.loc[:,('Battery_EV')]/outflow_df.loc[:,('Total')]) * 100
outflow_components_3.loc[:,'Other_powertrain_ICEV_rel'] = (outflow_components_3.loc[:,('Other_powertrain_ICEV')]/outflow_df.loc[:,('Total')]) * 100
outflow_components_3.loc[:,'Total'] = outflow_components_3.loc[:,('Battery_EV_rel')] + outflow_components_3.loc[:,('Other_powertrain_ICEV_rel')]

outflow_components_4.loc[:,'Battery_EV_rel'] = (outflow_components_4.loc[:,('Battery_EV')]/outflow_df.loc[:,('Total')]) * 100
outflow_components_4.loc[:,'Other_powertrain_ICEV_rel'] = (outflow_components_4.loc[:,('Other_powertrain_ICEV')]/outflow_df.loc[:,('Total')]) * 100
outflow_components_4.loc[:,'Total'] = outflow_components_4.loc[:,('Battery_EV_rel')] + outflow_components_4.loc[:,('Other_powertrain_ICEV_rel')]

outflow_components_5.loc[:,'Battery_EV_rel'] = (outflow_components_5.loc[:,('Battery_EV')]/outflow_df.loc[:,('Total')]) * 100
outflow_components_5.loc[:,'Other_powertrain_ICEV_rel'] = (outflow_components_5.loc[:,('Other_powertrain_ICEV')]/outflow_df.loc[:,('Total')]) * 100
outflow_components_5.loc[:,'Total'] = outflow_components_5.loc[:,('Battery_EV_rel')] + outflow_components_5.loc[:,('Other_powertrain_ICEV_rel')]

3  #################################
####################################
outflow_comp = outflow_components_1
#outflow_comp = outflow_components_2
#outflow_comp = outflow_components_3
#outflow_comp = outflow_components_4
#outflow_comp = outflow_components_5
###################################
###################################

#Defining of Entropy functions for production stage and for ELV stage (two versions cover merely the different input formats)
def Entropy_production(flow):
    M = flow.sum()
    c = flow/M
    M_global = 56455.19093
    m = M/M_global
    
    with errstate(divide='ignore'):
        log_c = np.log2(c)
        log_c[isneginf(log_c)] = 0
    
    H = -sum(m * c * log_c)
    return H

def Entropy(flow):
    flow = flow.sum()
    M = flow.sum()
    c = flow/M
    M_global = 56455.19093
    m = M/M_global
    
    with errstate(divide='ignore'):
        log_c = np.log2(c)
        log_c[isneginf(log_c)] = 0
    
    H = -sum(m * c * log_c)
    return H
#This is the highest entropy level one can achieve with the given car composition of EVs and ICEVs, when optimizing for Hmax
Hmax = 2.6258

# in case, component flows are to be set to zero, activate the code
#outflow_comp = pd.read_csv("_components_empty.csv", sep=';', index_col='Year')

In [ ]:
#Program that calculates the material flows, reuse and recycling flows. It also calculates the Relative Statistical
#Entropy per stage and year. 

#Initialization of recycling flows as variables with 0 value for the first year
recycled_Al = 0 
recycled_steel = 0
recycled_copper = 0
produced_vehicles = generic_df_1percent.mul([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], axis=0)

#placeholder so that first loop can be run
ev_batteries_f = (produced_vehicles.iloc[0] * 0) 
other_powertrain_components_f = (produced_vehicles.iloc[0] * 0)

#Initialization of recycling flows
recycled_Al_f = pd.read_csv('empty_df.csv', sep=';', index_col='Flow')
recycled_steel_f = pd.read_csv('empty_df.csv', sep=';', index_col='Flow')
recycled_copper_f = pd.read_csv('empty_df.csv', sep=';', index_col='Flow')
max_mass_flow = 17.4274

for i in range(41):
                
        #Take percentage-value for year i from use phase to ELV sector from the outflow-file generated in dynamic model
        EV_out = outflow_df.iloc[i, 3]
        ICEV_out = outflow_df.iloc[i, 4]
        G_out = (EV_out + ICEV_out)
        
        #absolute mass flows
        EV_out = EV_out * max_mass_flow
        ICEV_out = ICEV_out * max_mass_flow
        G_out = G_out * max_mass_flow
        year = outflow_df.index[i]
        
        try:
            if year < 2052: #2030
                Tks = Tks
            else:
                Tks = Tks2
        except:
            print("Error when setting Tks.")
        
        dismantled_ELVs_tk = Tks.iloc[0]
        materials_from_EV_motor_tk = Tks.iloc[1]
        exported_copper_tk = Tks.iloc[2]
        recycled_copper_tk = Tks.iloc[3]
        ev_motors_tk = Tks.iloc[4]
        slags_battery_recycling_tk = Tks.iloc[5]
        ev_batteries_exported_tk = Tks.iloc[6]
        ev_batteries_recycled_tk = Tks.iloc[7]
        heavy_fraction_tk = Tks.iloc[8]
        light_fraction_tk = Tks.iloc[9]
        exported_Fe_heavy_tk = Tks.iloc[10]
        domestic_Fe_tk = Tks.iloc[11]
        non_Fe_fraction_tk = Tks.iloc[12]
        shredder_fluff_1_tk = Tks.iloc[13]
        shredder_fluff_2_tk = Tks.iloc[14]
        domestic_Aluminum_tk = Tks.iloc[15]
        exported_mixed_fraction_tk = Tks.iloc[16]
        exported_Al_fraction_tk = Tks.iloc[17]
        exported_Fe_fraction_tk = Tks.iloc[18]
        residues_tk = Tks.iloc[19]
        al_fraction_tk = Tks.iloc[20]
        exported_steel_tk = Tks.iloc[21]
        slags_and_dusts_steel_tk = Tks.iloc[22]
        recycled_steel_tk = Tks.iloc[23]
        exported_Al_tk = Tks.iloc[24]
        slags_Al_tk = Tks.iloc[25]
        recycled_Al_tk = Tks.iloc[26]
        recycled_materials_tk = Tks.iloc[27]
        slags_regulated_components_tk = Tks.iloc[28]
        domestic_steel_from_decanning_tk = Tks.iloc[29]
        
        EV_in = inflow_df.iloc[i, 3]
        ICEV_in = inflow_df.iloc[i, 4]
        G_in = (EV_in + ICEV_in)
        
        #absolute mass flows
        EV_in = EV_in * max_mass_flow
        ICEV_in = ICEV_in * max_mass_flow
        G_in = G_in * max_mass_flow
        
        #outflows of EV components to Dismantling process
        EV_comp_out = outflow_comp.iloc[i, 3]
        ICEV_comp_out = outflow_comp.iloc[i, 4]
        
        #absolute mass flows
        EV_comp_out = EV_comp_out * max_mass_flow
        ICEV_comp_out = ICEV_comp_out * max_mass_flow
        
        #Multiplicate EV, ICEV with singel rows in df for production phase
        produced_vehicles = generic_df_1percent.mul([EV_in, EV_in, EV_in, G_in, G_in, G_in, G_in, G_in, 
                                             ICEV_in, ICEV_in, ICEV_in, ICEV_in], axis=0)
        
        #Multiplicate EV, ICEV with singel rows in df for ELV phase
        elvs_f = generic_df_1percent.mul([EV_out, EV_out, EV_out, G_out, G_out, G_out, G_out, G_out,
                                                   ICEV_out, ICEV_out, ICEV_out, ICEV_out], axis=0)
        
        #Component flows to ELV sector (spare-parts)
        EV_comp_out = outflow_comp.iloc[i, 3]
        ICEV_comp_out = outflow_comp.iloc[i, 4]
        
        #absolute mass flows
        EV_comp_out = EV_comp_out * max_mass_flow
        ICEV_comp_out = ICEV_comp_out * max_mass_flow
        
        components_f = generic_df_1percent.mul([EV_comp_out, 0, 0, 0, 0, 0, 0, 0, 0, 0, ICEV_comp_out, 0], axis=0)
        #components_f = generic_df_1percent.mul([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], axis=0)
        
        EV_battery_elv_comp = components_f.iloc[0]
        EV_motor_and_transmission_elv_comp = components_f.iloc[1]
        ICEV_other_powertrain_elv_comp = components_f.iloc[10]
        
        #Produced vehicles are used to calculate the material demand
        produced_vehicles = produced_vehicles + components_f
        
        #Offsetting spare parts production by reused components
        imported_mat = produced_vehicles      
        
        #######################################################################
        #PRODUCTION MODULE
        
        #Definition of each component in terms of its material composition for production stage
        EV_battery = produced_vehicles.iloc[0]
        EV_motor_and_transmission = produced_vehicles.iloc[1]
        Lead_battery_EV = produced_vehicles.iloc[2]
        Interrior_exterior = produced_vehicles.iloc[3]
        Tyres_wheels = produced_vehicles.iloc[4]
        Brakes = produced_vehicles.iloc[5]
        Chassis = produced_vehicles.iloc[6]
        Body_doors = produced_vehicles.iloc[7]
        Transmission = produced_vehicles.iloc[8]
        Engine = produced_vehicles.iloc[9]
        ICEV_other_powertrain = produced_vehicles.iloc[10]
        Lead_battery_ICEV = produced_vehicles.iloc[11]
        
        #######################################################################
        #REUSE MODULE
        
        if EV_battery_elv_comp.sum() > ev_batteries_f.sum().sum():  #this way, only spare part batteries are offset.
            EV_battery = EV_battery - ev_batteries_f.sum()
        else:
            EV_battery = EV_battery - EV_battery_elv_comp
            surplus_battery = (ev_batteries_f.sum() - EV_battery_elv_comp).abs()
            
        ##########################
        if ICEV_other_powertrain_elv_comp.sum() > other_powertrain_components_f.sum().sum():
            ICEV_other_powertrain = ICEV_other_powertrain - other_powertrain_components_f
        else:
            ICEV_other_powertrain = ICEV_other_powertrain*0
            surplus = (ICEV_other_powertrain - ICEV_other_powertrain_elv_comp).abs()
        
        ######################################################################
        #IMPORTS MODULE
        #Definition of imported flows
        imported_Fe = empty_df.loc[['imported_Fe'], :]
        imported_Steel = empty_df.loc[['imported_Steel'], :]
        imported_Plastic = empty_df.loc[['imported_Plastic'], :]
        imported_Cu = empty_df.loc[['imported_Cu'], :]
        imported_Glass = empty_df.loc[['imported_Glass'], :]
        imported_Al = empty_df.loc[['imported_Al'], :]
        imported_Al_cast = empty_df.loc[['imported_Al_cast'], :]
        imported_Paint = empty_df.loc[['imported_Paint'], :]
        imported_Rubber = empty_df.loc[['imported_Rubber'], :]
        imported_Carbon_black = empty_df.loc[['imported_Carbon_black'], :]
        imported_Pb = empty_df.loc[['imported_Pb'], :]
        imported_Ethylencarbonate = empty_df.loc[['imported_Ethylencarbonate'], :]
        imported_Graphite = empty_df.loc[['imported_Graphite'], :]
        imported_Nd = empty_df.loc[['imported_Nd'], :]
        imported_LiMn04 = empty_df.loc[['imported_LiMn04'], :]
        imported_LiPF6 = empty_df.loc[['imported_LiPF6'], :]
        
        ###Offsetting imports by recycled materials and reused components
        imported_Fe.loc['imported_Fe', 'Fe'] = imported_mat['Fe'].sum()
        imported_Steel.loc['imported_Steel', 'Steel'] = (imported_mat['Steel'].sum())
        imported_Plastic.loc['imported_Plastic', 'Plastic'] = imported_mat['Plastic'].sum()
        imported_Cu.loc['imported_Cu', 'Cu'] = imported_mat['Cu'].sum()
        imported_Glass.loc['imported_Glass', 'Glass'] = imported_mat['Glass'].sum()
        imported_Al.loc['imported_Al', 'Al'] = imported_mat['Al'].sum()
        imported_Al_cast.loc['imported_Al_cast', 'Al_cast'] = (imported_mat['Al_cast'].sum())
        imported_Paint.loc['imported_Paint', 'Paint'] = imported_mat['Paint'].sum()
        imported_Rubber.loc['imported_Rubber', 'Rubber'] = imported_mat['Rubber'].sum()
        imported_Carbon_black.loc['imported_Carbon_black', 'Carbon_black'] = imported_mat['Carbon_black'].sum()
        imported_Pb.loc['imported_Pb', 'Pb'] = imported_mat['Pb'].sum()
        imported_Ethylencarbonate.loc['imported_Ethylencarbonate', 'Ethylencarbonate'] = imported_mat['Ethylencarbonate'].sum()
        imported_Graphite.loc['imported_Graphite', 'Graphite'] = imported_mat['Graphite'].sum()
        imported_Nd.loc['imported_Nd', 'Nd'] = imported_mat['Nd'].sum()
        imported_LiMn04.loc['imported_LiMn04', 'LiMn04'] = imported_mat['LiMn04'].sum()
        imported_LiPF6.loc['imported_LiPF6', 'LiPF6'] = imported_mat['LiPF6'].sum()
        
        ######################################################################
        #RECYCLING MODULE
        
        #Uptake cascade for recycled_Al
        recycled_Al = recycled_Al_f.sum().sum()
        
        try:
            if produced_vehicles.loc['Engine', 'Al_cast'] > recycled_Al:
                imported_Al_cast.loc['imported_Al_cast', 'Al_cast'] = (produced_vehicles.loc['Engine', 'Al_cast'] - recycled_Al)
            else: 
                surplus_Al = recycled_Al - imported_Al_cast.loc['imported_Al_cast', 'Al_cast']
                recycled_Al = imported_Al_cast.loc['imported_Al_cast', 'Al_cast'] #recycl. Al equals Al categories that take up recyc. Al
                imported_Al_cast.loc['imported_Al_cast', 'Al_cast'] = 0
        except:
            print('Error in the uptake of recycled Aluminum!')
        
        #Uptake cascade for recycled_steel 
        recycled_steel = recycled_steel_f.sum().sum()
        
        try:
            if produced_vehicles.loc['Body_doors', 'Steel'] > recycled_steel:
                imported_Steel.loc['imported_Steel', 'Steel'] = imported_Steel.loc['imported_Steel', 'Steel'] - recycled_steel
            else:
                surplus_steel = recycled_steel - imported_Steel.loc['imported_Steel', 'Steel']
                recycled_steel = produced_vehicles.loc['Body_doors', 'Steel']
                imported_Steel.loc['imported_Steel', 'Steel'] = 0
        except:
            print('Error in the uptake of recycled Steel!')
        
        #Uptake cascade for recycled_copper
        recycled_copper = recycled_copper_f.sum().sum()

        try:
            if produced_vehicles.loc['EV_motor_and_transmission', 'Cu'] > recycled_copper:
                imported_Cu.loc['imported_Cu', 'Cu'] = imported_Cu.loc['imported_Cu', 'Cu'] - recycled_copper
            else:
                surplus_copper = recycled_copper - imported_Cu.loc['imported_Cu', 'Cu']
                recycled_copper = produced_vehicles.loc['EV_motor_and_transmission', 'Cu']

        except:
            print('Error in the uptake of recycled copper!')
              
        ######################################################################
        #MFA
        ######################################################################
        #ELV IMPORT TO ELV SYSTEM MODULE
        
        #Definition of each component in terms of its material composition for production stage       
        EV_battery_elv = elvs_f.iloc[0]
        EV_motor_and_transmission_elv = elvs_f.iloc[1]
        Lead_battery_EV_elv = elvs_f.iloc[2]
        Interrior_exterior_elv = elvs_f.iloc[3]
        Tyres_wheels_elv = elvs_f.iloc[4]
        Brakes_elv = elvs_f.iloc[5]
        Chassis_elv = elvs_f.iloc[6]
        Body_doors_elv = elvs_f.iloc[7]
        Transmission_elv = elvs_f.iloc[8]
        Engine_elv = elvs_f.iloc[9]
        ICEV_other_powertrain_elv = elvs_f.iloc[10]
        Lead_battery_ICEV_elv = elvs_f.iloc[11]   

        ############################################################
        ### # # # # # recycling rates for components (diversion to recycling processes)
        try:
            if year < 2052: #2025
                ev_battery_r = 0.45 
                ev_motors_r = 0.45  
                other_powertrain_to_other_powertrain_components = 0.45
                other_powertrain_to_regulated_components = other_powertrain_to_other_powertrain_components   #has to sum to one
                other_powertrain_to_dismantled_elvs = (1-(other_powertrain_to_regulated_components + other_powertrain_to_other_powertrain_components))

            else:
                ev_battery_r = 0.95
                ev_motors_r = 0.95
                
                other_powertrain_to_other_powertrain_components = 0.95
                other_powertrain_to_regulated_components = other_powertrain_to_other_powertrain_components   #has to sum to one
                other_powertrain_to_dismantled_elvs = (1-(other_powertrain_to_regulated_components+other_powertrain_to_other_powertrain_components))
                
        except:
            print("Error in component recycling loop.")
                
        ###Component diversion from ELVs
        ev_motors_f = elvs_f.mul([0, ev_motors_r, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], axis=0)
        ev_batteries_f = elvs_f.mul([ev_battery_r, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], axis=0)
        regulated_components_f = elvs_f.mul([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, other_powertrain_to_regulated_components, 0], axis=0)
        other_powertrain_components_f = elvs_f.mul([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, other_powertrain_to_other_powertrain_components, 0], axis=0)

        #Rest of components from ELVs to dismantled elvs
        EV_motor_and_transmission_elv_car = (EV_motor_and_transmission_elv * (1-ev_motors_r))
        EV_battery_elv_car = (EV_battery_elv * (1-ev_battery_r))
        ICEV_other_powertrain_elv_car = (ICEV_other_powertrain_elv * other_powertrain_to_dismantled_elvs)
        
        #Test for mass conservation
        dismantled_elvs_f = elvs_f.mul([(1-ev_battery_r), (1-ev_motors_r), 1, 1, 1, 1, 1, 1, 1, 1, other_powertrain_to_dismantled_elvs, 1], axis=0)
        
        ########################################
        ###Component diversion from spare parts
        ev_motors_components_r = 0.95
        ev_battery_components_r = 0.95
        
        other_powertrain_components_to_dismantled_elvs = 0.2
        other_powertrain_components_to_regulated_components = 0.3
        other_powertrain_components_to_other_powertrain_components = 0.5
        
        components_to_ev_batteries_f = components_f.mul([ev_battery_components_r, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], axis=0)
        components_to_ev_motors_f = components_f.mul([0, ev_motors_components_r, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], axis=0)
        components_to_regulated_components_f = components_f.mul([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, other_powertrain_components_to_regulated_components, 0], axis=0)
        components_to_other_powertrain_components_f = components_f.mul([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, other_powertrain_components_to_other_powertrain_components, 0], axis=0)
        
        ev_motors_f = ev_motors_f + components_to_ev_motors_f
        regulated_components_f = regulated_components_f + components_to_regulated_components_f
        ev_batteries_f = ev_batteries_f + components_to_ev_batteries_f
        other_powertrain_components_f = other_powertrain_components_f + components_to_other_powertrain_components_f
        
        #Rest of components to dismantled elvs    
        EV_motor_and_transmission_elv = EV_motor_and_transmission_elv_car + (EV_motor_and_transmission_elv_comp * (1-ev_motors_components_r))
        EV_battery_elv = EV_battery_elv_car + (EV_battery_elv_comp * (1-ev_battery_components_r))
        ICEV_other_powertrain_elv = ICEV_other_powertrain_elv_car + (ICEV_other_powertrain_elv_comp * other_powertrain_components_to_dismantled_elvs)
        
        #Test for mass conserv.
        components_to_dismantled_elvs_f = components_f.mul([(1-ev_battery_components_r), (1-ev_motors_components_r), 1, 1, 1, 1, 1, 1, 1, 1, other_powertrain_components_to_dismantled_elvs, 1], axis=0)
        dismantled_elvs_f = dismantled_elvs_f + components_to_dismantled_elvs_f

        ##################################################################
        ##################################################################
        ##Recycling
        #Calculation of flows by multiplication of each flow with Tks

        #Process: EV motor recycling
        exported_copper_f = ev_motors_f * exported_copper_tk ###exprts
        recycled_copper_f = ev_motors_f * recycled_copper_tk 
        materials_from_ev_motor_f = ev_motors_f * materials_from_EV_motor_tk ###exprts
        ev_motors_reused_f = ev_motors_f * ev_motors_tk
        
        #Process: Hammer mill separation
        heavy_fraction_f = dismantled_elvs_f * heavy_fraction_tk
        light_fraction_f = dismantled_elvs_f * light_fraction_tk

        #Process: Processing of regulated components
        recycled_materials_f = regulated_components_f * recycled_materials_tk
        slags_regulated_components_f = regulated_components_f * slags_regulated_components_tk
        domestic_steel_from_decanning_f = regulated_components_f * domestic_steel_from_decanning_tk
        
        #Process: Battery recycling
        ev_batteries_exported_f = ev_batteries_f * ev_batteries_exported_tk
        slags_battery_recycling_f = ev_batteries_f * slags_battery_recycling_tk
        ev_batteries_recycled_f = ev_batteries_f * ev_batteries_recycled_tk
        
        #Process: Magnetic separation
        exported_Fe_heavy_f = heavy_fraction_f * exported_Fe_heavy_tk
        domestic_Fe_f = heavy_fraction_f * domestic_Fe_tk
        non_Fe_fraction_f = heavy_fraction_f * non_Fe_fraction_tk

        #Process: Light fraction processing (to do: fines have been integrated with one of the two shredder fluffs, update in STAN)
        shredder_fluff_1_f = light_fraction_f * shredder_fluff_1_tk
        shredder_fluff_2_f = light_fraction_f * shredder_fluff_2_tk
        domestic_Aluminum_f = light_fraction_f * domestic_Aluminum_tk
        
        #Process: Heavy media processing
        exported_mixed_fraction_f = non_Fe_fraction_f * exported_mixed_fraction_tk
        exported_Al_fraction_f = non_Fe_fraction_f * exported_Al_fraction_tk
        exported_Fe_fraction_f = non_Fe_fraction_f * exported_Fe_fraction_tk
        residues_f = non_Fe_fraction_f * residues_tk
        #heavy_media_f = non_Fe_fraction_f * heavy_media_tk
        al_fraction_f = non_Fe_fraction_f * al_fraction_tk

        #Aggregation of domestic Al from light fraction processing and Al fraction from heavy media processing
        domestic_Aluminum_aggregated_f = domestic_Aluminum_f + al_fraction_f

        #Aggregation of heavy media and shredder fluff 2 from light fraction processing, which are directed to energy recovery
        #heavy_media_ASR_combined_f = shredder_fluff_2_f + heavy_media_f
        
        #Process: Domestic steel production (Attention, two df have to be aggregated (Domestic steel from decanning and Domestic Fe))
        input_domestic_steel_production_f = domestic_Fe_f + domestic_steel_from_decanning_f
        exported_steel_f = input_domestic_steel_production_f * exported_steel_tk
        slags_and_dusts_steel_f = input_domestic_steel_production_f * slags_and_dusts_steel_tk
        recycled_steel_f = input_domestic_steel_production_f * recycled_steel_tk

        #Process: Domestic Al production (Attention, two df have to be aggregated ...)
        exported_Al_f = domestic_Aluminum_aggregated_f * exported_Al_tk
        slags_Al_f = domestic_Aluminum_aggregated_f * slags_Al_tk
        recycled_Al_f = domestic_Aluminum_aggregated_f * recycled_Al_tk
        
        recycled_Al = recycled_Al_f.sum().sum()
        recycled_steel = recycled_steel_f.sum().sum()
        recycled_copper = recycled_copper_f.sum().sum()

        ######################################################################
        
        #Define import flow variables for production stage and store in list for iteration in H-calculation Entropy-function
        import_flow_variables = [imported_Fe, imported_Steel, imported_Plastic, imported_Cu, imported_Glass, imported_Al,
                                imported_Al_cast, imported_Paint, imported_Rubber, imported_Carbon_black, imported_Pb, 
                                imported_Ethylencarbonate, imported_Graphite, imported_Nd, imported_LiMn04, imported_LiPF6]
        
        import_flow_names = ['imported_Fe', 'imported_Steel', 'imported_Plastic', 'imported_Cu', 'imported_Glass', 
                             'imported_Al', 'imported_Al_cast', 'imported_Paint', 'imported_Rubber', 'imported_Carbon_black',
                             'imported_Pb', 'imported_Ethylencarbonate', 'imported_Graphite', 'imported_Nd', 
                             'imported_LiMn04', 'imported_LiPF6']
        
        
        #Define flow variables for ELV stage and store in list for iteration in H-calculation Entropy-function
        flow_variables = [heavy_fraction_f, light_fraction_f, exported_Fe_heavy_f, domestic_Fe_f, 
                      non_Fe_fraction_f , shredder_fluff_1_f, shredder_fluff_2_f, domestic_Aluminum_f, 
                      exported_mixed_fraction_f, exported_Al_fraction_f, exported_Fe_fraction_f, residues_f, 
                      al_fraction_f, exported_steel_f, slags_and_dusts_steel_f, recycled_steel_f, 
                      exported_Al_f, slags_Al_f, recycled_Al_f, 
                      components_f, exported_copper_f, recycled_copper_f, materials_from_ev_motor_f, ev_motors_reused_f, 
                      domestic_steel_from_decanning_f, slags_regulated_components_f, recycled_materials_f, ev_batteries_exported_f, 
                      slags_battery_recycling_f, ev_batteries_recycled_f]
        
        flow_names = ['heavy_fraction_f', 'light_fraction_f', 'exported_Fe_heavy_f', 'domestic_Fe_f', 
                      'non_Fe_fraction_f' , 'shredder_fluff_1_f', 'shredder_fluff_2_f', 'domestic_Aluminum_f', 
                      'exported_mixed_fraction_f', 'exported_Al_fraction_f', 'exported_Fe_fraction_f', 'residues_f', 
                      'al_fraction_f', 'exported_steel_f', 'slags_and_dusts_steel_f', 'recycled_steel_f', 
                      'exported_Al_f', 'slags_Al_f', 'recycled_Al_f', 
                      'components_f', 'exported_copper_f', 'recycled_copper_f', 'materials_from_ev_motor_f', 'ev_motors_reused_f', 
                      'domestic_steel_from_decanning_f', 'slags_regulated_components_f', 'recycled_materials_f', 'ev_batteries_exported_f', 
                      'slags_battery_recycling_f', 'ev_batteries_recycled_f']
        
        components_f = components_f.sum()
        ev_motors_f = ev_motors_f.sum()
        ev_motors_reused_f = ev_motors_reused_f.sum()
        ev_batteries_f_agg = ev_batteries_f.sum()
        other_powertrain_components_f = other_powertrain_components_f.sum()
        dismantled_elvs_f = dismantled_elvs_f.sum()
        regulated_components_f = regulated_components_f.sum() 
        
        component_variables = [EV_battery, EV_motor_and_transmission, Lead_battery_EV, Interrior_exterior, 
                       Tyres_wheels, Brakes, Chassis, Body_doors, Transmission, Engine, 
                       ICEV_other_powertrain, Lead_battery_ICEV, EV_battery_elv, EV_motor_and_transmission_elv, Lead_battery_EV_elv, 
                       Interrior_exterior_elv, Tyres_wheels_elv, Brakes_elv, Chassis_elv, Body_doors_elv,
                       Transmission_elv, Engine_elv, ICEV_other_powertrain_elv, Lead_battery_ICEV_elv, 
                        other_powertrain_components_f, regulated_components_f, ev_motors_f, ev_motors_reused_f, ev_batteries_f_agg, components_f]

        component_names = ['EV_battery', 'EV_motor_and_transmission', 'Lead_battery_EV', 'Interrior_exterior', 
                   'Tyres_wheels','Brakes', 'Chassis', 'Body_doors', 'Transmission', 'Engine', 
                   'ICEV_other_powertrain', 'Lead_battery_ICEV', 'EV_battery_elv', 'EV_motor_and_transmission_elv', 
                   'Lead_battery_EV_elv', 'Interrior_exterior_elv', 'Tyres_wheels_elv','Brakes_elv', 'Chassis_elv',
                   'Body_doors_elv', 'Transmission_elv', 'Engine_elv', 'ICEV_other_powertrain_elv', 'Lead_battery_ICEV_elv', 
                    'other_powertrain_components_f', 'regulated_components_f', 'ev_motors_f','ev_motors_reused_f', 'ev_batteries_f_agg', 'components_f'] 
          
        ####### # # 
        #calculating imported materials after subtracting recycled materials
        imp_mat=[]
        for imp in import_flow_variables:
            g = imp.sum().sum()
            imp_mat.append(g)
        imp_mat_effective = np.sum(imp_mat)
        
        #############################################################
        #ENTROPY CALCULATION
        
        #creating dataframe with flow names for IMPORT PHASE, calculation of Entropy for each flow
        cols = ['Flow', 'H']
        results_imports_df = pd.DataFrame(columns=cols, index=range(len(import_flow_variables)))
        for f, g in zip(range(len(import_flow_names)), range(len(import_flow_variables))):
            try:
                results_imports_df.loc[f].Flow = import_flow_names[f]
                results_imports_H = Entropy(import_flow_variables[g])
                results_imports_df.loc[g].H = results_imports_H
            except:
                print('Error in the second loop of imports!')              
        
        #creating dataframe with flow names for PRODUCTION PHASE, calculation of Entropy for each flow
        cols = ['Flow', 'H']
        results_production_df = pd.DataFrame(columns=cols, index=range(len(component_variables)))
        
        for f, g in zip(range(len(component_names)), range(len(component_variables))):
            try:
                results_production_df.loc[f].Flow = component_names[f]
                result_production_H = Entropy_production(component_variables[g])
                results_production_df.loc[g].H = result_production_H             
            except:
                print('Error in the second loop of production!')  
                
        results_production_df = results_production_df.fillna(0) 
        
        #creating dataframe with flow names for ELV PHASE, calculation of Entropy for each flow         
        results_df = pd.DataFrame(columns=cols, index=range(len(flow_variables)))
        
        for f, g in zip(range(len(flow_names)), range(len(flow_variables))):
            try:      
                results_df.loc[f].Flow = flow_names[f]
                result_H = Entropy(flow_variables[g])
                results_df.loc[g].H = result_H
            except:
                print('Error in the second loop of ELV flows!')    
        
        results_df['Hrel'] = results_df['H']/Hmax
        results_production_df['Hrel'] = results_production_df['H']/Hmax
        results_imports_df['Hrel'] = results_imports_df['H']/Hmax
        
        results_imports_df['Hrel'] = results_imports_df['H']/Hmax

        #append production, imports and ELV data
        results_df = results_df.append(results_production_df)
        results_df = results_df.append(results_imports_df)
        
        #####    #####
        # Aggregation of Hrel-values for stage0 (import phase)
        results_df.loc[results_df.Flow == 'imported_Fe', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Steel', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Plastic', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Cu', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Glass', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Al', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Al_cast', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Paint', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Rubber', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Carbon_black', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Pb', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Ethylencarbonate', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Graphite', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_Nd', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_LiMn04', 'Stage0'] = 'Stage 0'
        results_df.loc[results_df.Flow == 'imported_LiPF6', 'Stage0'] = 'Stage 0'
        
        # Aggregation of Hrel-values for stage1 (production phase)
        results_df.loc[results_df.Flow == 'EV_battery', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'EV_motor_and_transmission', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'Lead_battery_EV', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'Interrior_exterior', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'Tyres_wheels', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'Brakes', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'Chassis', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'Body_doors', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'Transmission', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'Engine', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'ICEV_other_powertrain', 'Stage1'] = 'Stage 1'
        results_df.loc[results_df.Flow == 'Lead_battery_ICEV', 'Stage1'] = 'Stage 1'
       
        # Aggregation of Hrel-values for stage2
        #components:
        results_df.loc[results_df.Flow == 'ev_batteries_f_agg', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'ev_motors_f', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'regulated_components_f', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'other_powertrain_components_f', 'Stage2'] = 'Stage 2'
        #elv:
        results_df.loc[results_df.Flow == 'EV_battery_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'EV_motor_and_transmission_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'Lead_battery_EV_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'Interrior_exterior_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'Tyres_wheels_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'Brakes_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'Chassis_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'Body_doors_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'Transmission_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'Engine_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'ICEV_other_powertrain_elv', 'Stage2'] = 'Stage 2'
        results_df.loc[results_df.Flow == 'Lead_battery_ICEV_elv', 'Stage2'] = 'Stage 2'
        
        # Aggregation of Hrel-values for stage3
        results_df.loc[results_df.Flow == 'ev_batteries_f_agg', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'ev_motors_f', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'regulated_components_f', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'other_powertrain_components_f', 'Stage3'] = 'Stage 3'
        #dismantled_elvs_f:
        results_df.loc[results_df.Flow == 'EV_battery_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'EV_motor_and_transmission_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'Lead_battery_EV_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'Interrior_exterior_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'Tyres_wheels_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'Brakes_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'Chassis_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'Body_doors_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'Transmission_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'Engine_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'ICEV_other_powertrain_elv', 'Stage3'] = 'Stage 3'
        results_df.loc[results_df.Flow == 'Lead_battery_ICEV_elv', 'Stage3'] = 'Stage 3'
        
        # Aggregation of Hrel-values for stage4
        results_df.loc[results_df.Flow == 'exported_copper_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'recycled_copper_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'materials_from_ev_motor_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'ev_motors_reused_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'light_fraction_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'heavy_fraction_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'recycled_materials_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'slags_regulated_components_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'domestic_steel_from_decanning_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'ev_batteries_exported_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'slags_battery_recycling_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'ev_batteries_recycled_f', 'Stage4'] = 'Stage 4'
        results_df.loc[results_df.Flow == 'other_powertrain_components_f', 'Stage4'] = 'Stage 4'
        
        # Aggregation of Hrel-values for stage5
        results_df.loc[results_df.Flow == 'exported_copper_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'recycled_copper_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'materials_from_ev_motor_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'ev_motors_reused_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'exported_Fe_heavy_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'domestic_Fe_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'non_Fe_fraction_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'shredder_fluff_1_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'shredder_fluff_2_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'domestic_Aluminum_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'recycled_materials_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'slags_regulated_components_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'domestic_steel_from_decanning_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'ev_batteries_exported_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'slags_battery_recycling_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'ev_batteries_recycled_f', 'Stage5'] = 'Stage 5'
        results_df.loc[results_df.Flow == 'other_powertrain_components_f', 'Stage5'] = 'Stage 5'    
        
        # Aggregation of Hrel-values for stage6 
        results_df.loc[results_df.Flow == 'exported_copper_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'recycled_copper_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'materials_from_ev_motor_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'ev_motors_reused_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'exported_Fe_heavy_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'domestic_Fe_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'domestic_Aluminum_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'shredder_fluff_1_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'shredder_fluff_2_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'exported_mixed_fraction_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'exported_Al_fraction_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'exported_Fe_fraction_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'residues_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'al_fraction_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'recycled_materials_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'slags_regulated_components_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'domestic_steel_from_decanning_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'ev_batteries_exported_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'slags_battery_recycling_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'ev_batteries_recycled_f', 'Stage6'] = 'Stage 6'
        results_df.loc[results_df.Flow == 'other_powertrain_components_f', 'Stage6'] = 'Stage 6'
        
        # Aggregation of Hrel-values for stage7
        results_df.loc[results_df.Flow == 'exported_copper_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'recycled_copper_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'materials_from_ev_motor_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'ev_motors_reused_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'exported_Fe_heavy_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'shredder_fluff_1_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'exported_mixed_fraction_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'exported_Al_fraction_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'exported_Fe_fraction_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'residues_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'exported_steel_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'slags_and_dusts_steel_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'recycled_steel_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'exported_Al_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'slags_Al_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'recycled_Al_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'recycled_materials_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'slags_regulated_components_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'ev_batteries_exported_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'slags_battery_recycling_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'ev_batteries_recycled_f', 'Stage7'] = 'Stage 7'
        results_df.loc[results_df.Flow == 'other_powertrain_components_f', 'Stage7'] = 'Stage 7'
        
        #Aggregation of Hrel for each flow to Hrel per stage 
        results_stage_df0 = results_df.groupby(['Stage0']).sum()
        results_stage_df0 = results_stage_df0[['Hrel']]
        results_stage_df1 = results_df.groupby(['Stage1']).sum()
        results_stage_df1 = results_stage_df1[['Hrel']]
        results_stage_df2 = results_df.groupby(['Stage2']).sum()
        results_stage_df2 = results_stage_df2[['Hrel']]
        results_stage_df3 = results_df.groupby(['Stage3']).sum()
        results_stage_df3 = results_stage_df3[['Hrel']]
        results_stage_df4 = results_df.groupby(['Stage4']).sum()
        results_stage_df4 = results_stage_df4[['Hrel']]
        results_stage_df5 = results_df.groupby(['Stage5']).sum()
        results_stage_df5 = results_stage_df5[['Hrel']]
        results_stage_df6 = results_df.groupby(['Stage6']).sum()
        results_stage_df6 = results_stage_df6[['Hrel']]
        results_stage_df7 = results_df.groupby(['Stage7']).sum()
        results_stage_df7 = results_stage_df7[['Hrel']]
        
        #Mass balance check
        flows_stage_0 = [imported_Fe, imported_Steel, imported_Plastic, imported_Cu, imported_Glass, imported_Al, 
                         imported_Al_cast, imported_Paint, imported_Rubber, imported_Carbon_black, imported_Pb,
                         imported_Ethylencarbonate, imported_Graphite, imported_Nd, imported_LiMn04, imported_LiPF6]
        
        flows_stage_1 = [EV_battery, EV_motor_and_transmission, Lead_battery_EV, Interrior_exterior, Tyres_wheels, Brakes, 
                         Chassis, Body_doors, Transmission, Engine, ICEV_other_powertrain, Lead_battery_ICEV]

        flows_stage_2 = [EV_battery_elv, EV_motor_and_transmission_elv, Lead_battery_EV_elv, Interrior_exterior_elv,
                        Tyres_wheels_elv, Brakes_elv, Chassis_elv, Body_doors_elv, Transmission_elv, Engine_elv,
                        ICEV_other_powertrain_elv, Lead_battery_ICEV_elv, ev_motors_f, regulated_components_f,
                        ev_batteries_f_agg, other_powertrain_components_f]     
        
        flows_stage_3 = [EV_battery_elv, EV_motor_and_transmission_elv, Lead_battery_EV_elv, Interrior_exterior_elv,
                        Tyres_wheels_elv, Brakes_elv, Chassis_elv, Body_doors_elv, Transmission_elv, Engine_elv,
                        ICEV_other_powertrain_elv, Lead_battery_ICEV_elv, ev_motors_f, regulated_components_f,
                        ev_batteries_f_agg, other_powertrain_components_f]
        
        flows_stage_4 = [exported_copper_f, recycled_copper_f, ev_motors_reused_f, materials_from_ev_motor_f, 
                         light_fraction_f, 
                         heavy_fraction_f, 
                         recycled_materials_f, slags_regulated_components_f, domestic_steel_from_decanning_f, 
                         ev_batteries_exported_f, slags_battery_recycling_f, 
                         ev_batteries_recycled_f, other_powertrain_components_f]
        
        flows_stage_5 = [exported_copper_f, recycled_copper_f, ev_motors_reused_f, materials_from_ev_motor_f, 
                         exported_Fe_heavy_f, domestic_Fe_f, non_Fe_fraction_f,  
                         domestic_Aluminum_f, shredder_fluff_1_f, shredder_fluff_2_f,
                         recycled_materials_f, slags_regulated_components_f, domestic_steel_from_decanning_f,
                         ev_batteries_exported_f, slags_battery_recycling_f, 
                         ev_batteries_recycled_f, other_powertrain_components_f]
        
        flows_stage_6 = [exported_copper_f, recycled_copper_f, ev_motors_reused_f, materials_from_ev_motor_f, 
                         exported_Fe_heavy_f, domestic_Fe_f, 
                         domestic_Aluminum_f, shredder_fluff_1_f, shredder_fluff_2_f, 
                         exported_mixed_fraction_f, exported_Al_fraction_f, exported_Fe_fraction_f, residues_f, al_fraction_f, 
                         recycled_materials_f, slags_regulated_components_f, domestic_steel_from_decanning_f, 
                         ev_batteries_exported_f, slags_battery_recycling_f, 
                         ev_batteries_recycled_f, other_powertrain_components_f]
        
        flows_stage_7 = [exported_copper_f, recycled_copper_f, ev_motors_reused_f, materials_from_ev_motor_f, 
                         exported_Fe_heavy_f, shredder_fluff_1_f, shredder_fluff_2_f,
                         exported_mixed_fraction_f, exported_Al_fraction_f, exported_Fe_fraction_f, residues_f, 
                         exported_steel_f, slags_and_dusts_steel_f, recycled_steel_f, 
                         exported_Al_f, slags_Al_f, recycled_Al_f,
                         recycled_materials_f, slags_regulated_components_f, 
                         ev_batteries_exported_f, slags_battery_recycling_f, ev_batteries_recycled_f, 
                         other_powertrain_components_f]
        
        production_stages = [flows_stage_0, flows_stage_1]
        elv_stages = [flows_stage_2, flows_stage_3, flows_stage_4, flows_stage_5, flows_stage_6, flows_stage_7]
        
        s = 0
        for stage in production_stages:
            M_sum = 0
            n = 0
            s = s + 1
            for i, df in enumerate(stage):
                M = stage[i].values.sum().sum()
                M_sum = M_sum + M
                n = n + 1
        
        s = 2
        for stage in elv_stages:
            M_sum = 0
            n = 0
            s = s + 1
            for i, df in enumerate(stage):
                M = stage[i].values.sum().sum()
                M_sum = M_sum + M
                n = n + 1

        #Append Hrel values to one dataframe:
        results_appended = results_stage_df0
        results_appended = results_appended.append(results_stage_df1)
        results_appended = results_appended.append(results_stage_df2)
        results_appended = results_appended.append(results_stage_df3)
        results_appended = results_appended.append(results_stage_df4)
        results_appended = results_appended.append(results_stage_df5)
        results_appended = results_appended.append(results_stage_df6)
        results_appended = results_appended.append(results_stage_df7)
        
        results_to_append = results_appended.assign(year=year)
        results_appended_df = results_appended_df.append(results_to_append)
        

In [ ]:
#################################################
#############################################################
#SAVING RESULTS TO CSV FILES
results_appended_df = results_appended_df.iloc[1:]
#results_appended_df.to_csv('Results 1 S70.csv', sep=';')
#results_appended_df.to_csv('Results 2 S100.csv', sep=';')

#single CE strategy effects
#results_appended_df.to_csv('Results 3 S70D.csv', sep=';')
#results_appended_df.to_csv('Results 4 S70D-LFT.csv', sep=';')
#results_appended_df.to_csv('Results 5 S70D-REU.csv', sep=';')
#results_appended_df.to_csv('Results 6 S70D-REC.csv', sep=';')

#combined CE strategy effects
#results_appended_df.to_csv('Results 7 S70-REU-REC.csv', sep=';')
#results_appended_df.to_csv('Results 8 S70D-REU-REC.csv', sep=';')
#results_appended_df.to_csv('Results 9 S70D-REU-REC-LFT.csv', sep=';')

#results_appended_df.to_csv('Results 10 S70-REU.csv', sep=';')
#results_appended_df.to_csv('Results 11 S70-REC.csv', sep=';')

#results_appended_df.to_csv('Results 12 S100D-REU-REC-LFT.csv', sep=';')

print(results_appended_df.shape)
results_appended_df

In [ ]:
#Preparation of dataframe for plots - Scenarios including Production (no mining stage)
S70 = pd.read_csv("Results 1 S70.csv", sep=';') 
S100 = pd.read_csv("Results 2 S100.csv", sep=';') 
S70D = pd.read_csv("Results 3 S70D.csv", sep=';') 

S70D_LFT = pd.read_csv("Results 4 S70D-LFT.csv", sep=';') 
S70D_REU = pd.read_csv("Results 5 S70D-REU.csv", sep=';') 
S70D_REC = pd.read_csv("Results 6 S70D-REC.csv", sep=';') 

S70_REU_REC = pd.read_csv("Results 7 S70-REU-REC.csv", sep=';') 
S70D_REU_REC = pd.read_csv("Results 8 S70D-REU-REC.csv", sep=';') 
S70D_REU_REC_LFT = pd.read_csv("Results 9 S70D-REU-REC-LFT.csv", sep=';') 

S70_REU = pd.read_csv("Results 10 S70-REU.csv", sep=';') 
S70_REC = pd.read_csv("Results 11 S70-REC.csv", sep=';')
S100D_REU_REC_LFT = pd.read_csv("Results 12 S100D-REU-REC-LFT.csv", sep=';') 

#combine data of different dataframes into one df
data = S70
data['S100'] = S100['Hrel'].values
data['S70D'] = S70D['Hrel'].values

data['S70D-LFT'] = S70D_LFT['Hrel'].values
data['S70D-REU'] = S70D_REU['Hrel'].values
data['S70D-REC'] = S70D_REC['Hrel'].values

data['S70-REU-REC'] = S70_REU_REC['Hrel'].values
data['S70D-REU-REC'] = S70D_REU_REC['Hrel'].values
data['S70D-REU-REC-LFT'] = S70D_REU_REC_LFT['Hrel'].values

data['S70-REU'] = S70_REU['Hrel'].values
data['S70-REC'] = S70_REC['Hrel'].values
data['S100D-REU-REC-LFT'] = S100D_REU_REC_LFT['Hrel'].values

#remane column names
data.rename(columns={'Unnamed: 0':'Stage'}, inplace=True)
data.rename(columns={'Hrel':'S70'}, inplace=True)

#rearrange order of columns
#old working
data_reuse = data[['Stage', 'year', 'S70', 'S100', 'S70D', 
                   'S70D-LFT', 'S70D-REU', 'S70D-REC', 
                   'S70-REU-REC', 'S70D-REU-REC', 'S70D-REU-REC-LFT', 
                   'S70-REU', 'S70-REC', 'S100D-REU-REC-LFT']]

data_disaggr = data_reuse

data_production = data_disaggr[data_disaggr.Stage.str.contains('Stage 1')]
data_ELV_with_stage0 = data_disaggr[~data_disaggr.Stage.str.contains('Stage 1')]
data_ELV = data_ELV_with_stage0[~data_ELV_with_stage0.Stage.str.contains('Stage 0')]

#working
data_ELV_values = data_ELV.loc[:,('S70', 'S100', 'S70D', 
                                  'S70D-LFT', 'S70D-REU', 'S70D-REC', 
                                  'S70-REU-REC', 'S70D-REU-REC', 'S70D-REU-REC-LFT', 
                                  'S70-REU', 'S70-REC', 'S100D-REU-REC-LFT')]

##########################
#Plot combined dataframes to compare reuse and no reuse cases
results_single_year1 = data_reuse.iloc[320:]
#results_single_year1 = data_reuse.iloc[160:]

plt.figure(figsize=(15,6))
ax = plt.gca()
results_single_year1.plot(y='S70', label='S70', color='tab:blue', linewidth=3,  ax=ax)
results_single_year1.plot(y='S100', label='S100', color='tab:red', linewidth=3, ax=ax)
results_single_year1.plot(y='S70-REU', label='S70-REU', color='tab:gray', linewidth=3, ax=ax)
results_single_year1.plot(y='S70-REU-REC', label='S70-REU-REC', color='tab:orange', linewidth=3, ax=ax)

results_single_year1.plot(y='S70D', label='S70D', color='tab:blue', dashes=[3, 0.5], linewidth=3, ax=ax)
results_single_year1.plot(y='S70D-REU', label='S70D-REU', color='tab:gray', dashes=[3, 0.5], linewidth=3, ax=ax)
results_single_year1.plot(y='S70D-REU-REC', label='S70D-REU-REC', color='tab:orange', dashes=[3, 0.5], linewidth=3, ax=ax)

#results_single_year1.plot(y='S70-REC', label='S70-REC', color='tab:green', linewidth=3, ax=ax)
#results_single_year1.plot(y='S70D-REC', label='S70D-REC', color='tab:green', dashes=[3, 0.5], linewidth=3, ax=ax)

results_single_year1.plot(y='S70D-LFT', label='S70D-LFT', color='tab:brown', dashes=[3, 0.5], linewidth=3, ax=ax)
results_single_year1.plot(y='S70D-REU-REC-LFT', label='S70D-REU-REC-LFT', color='dimgray', dashes=[1, 0.5], linewidth=3, ax=ax)
results_single_year1.plot(y='S100D-REU-REC-LFT', label='S100D-REU-REC-LFT', color='tab:red', dashes=[1, 0.5], linewidth=3, ax=ax)

ax.grid(True)
ax.set_ylabel("RSE", fontsize=14)
ax.legend(loc=2, prop={'size': 12})

plt.savefig('RSE evolution for multiple years.png', dpi=360)
#############################################################
# Graphs representing dH in ELV sector (differences in H between stages)
data_ELV_year = data_ELV.loc[:,('year')]
data_ELV_stage = data_ELV.loc[:,('Stage')]
data_ELV_stage_year = data_ELV.loc[:, ('Stage', 'year')]

#the order of the lines is important. It has to be identical with production data frame.
data_ELV_stage_year.loc[:, ('S70')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S70')].diff().fillna(0).abs()
data_ELV_stage_year.loc[:, ('S100')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S100')].diff().fillna(0).abs()
data_ELV_stage_year.loc[:, ('S70D')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S70D')].diff().fillna(0).abs()

data_ELV_stage_year.loc[:, ('S70D-LFT')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S70D-LFT')].diff().fillna(0).abs()
data_ELV_stage_year.loc[:, ('S70D-REU')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S70D-REU')].diff().fillna(0).abs()
data_ELV_stage_year.loc[:, ('S70D-REC')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S70D-REC')].diff().fillna(0).abs()

data_ELV_stage_year.loc[:, ('S70-REU-REC')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S70-REU-REC')].diff().fillna(0).abs()
data_ELV_stage_year.loc[:, ('S70D-REU-REC')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S70D-REU-REC')].diff().fillna(0).abs()
data_ELV_stage_year.loc[:, ('S70D-REU-REC-LFT')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S70D-REU-REC-LFT')].diff().fillna(0).abs()

data_ELV_stage_year.loc[:, ('S70-REU')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S70-REU')].diff().fillna(0).abs()
data_ELV_stage_year.loc[:, ('S70-REC')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S70-REC')].diff().fillna(0).abs()
data_ELV_stage_year.loc[:, ('S100D-REU-REC-LFT')] = data_ELV.groupby(data_ELV.loc[:,('year')])[('S100D-REU-REC-LFT')].diff().fillna(0).abs()

data_dH_ELV_cum = data_ELV_stage_year.groupby('year').sum()

plt.figure(figsize=(15,6))
ax = plt.gca()

data_dH_ELV_cum.plot(y='S70', label='S70', color='tab:blue', linewidth=4,  ax=ax)
data_dH_ELV_cum.plot(y='S100', label='S100', color='tab:red', linewidth=4, ax=ax)
data_dH_ELV_cum.plot(y='S70-REU', label='S70-REU', color='grey', linewidth=4, ax=ax)
data_dH_ELV_cum.plot(y='S70-REU-REC', label='S70-REU-REC', color='tab:orange', linewidth=4, ax=ax)

data_dH_ELV_cum.plot(y='S70D', label='S70D', color='tab:blue', dashes=[3, 0.5], linewidth=4, ax=ax)
data_dH_ELV_cum.plot(y='S70D-REU', label='S70D-REU', color='grey', dashes=[3, 0.5], linewidth=4, ax=ax)
data_dH_ELV_cum.plot(y='S70D-REU-REC', label='S70D-REU-REC', color='tab:orange', dashes=[3, 0.5], linewidth=4, ax=ax)

data_dH_ELV_cum.plot(y='S70D-LFT', label='S70D-LFT', color='tab:brown', dashes=[1, 0.5], linewidth=4, ax=ax)
data_dH_ELV_cum.plot(y='S70D-REU-REC-LFT', label='S70D-REU-REC-LFT', color='dimgray', dashes=[1, 0.5], linewidth=4, ax=ax)
data_dH_ELV_cum.plot(y='S100D-REU-REC-LFT', label='S100D-REU-REC-LFT', color='tab:red', dashes=[1, 0.5], linewidth=4, ax=ax)

ax.set_xlabel("Year", fontsize=12)
ax.set_ylabel(r'$\Delta RSE_{cum}$', fontsize=14)
ax.grid(True)
ax.set_ylim([0, 0.3])
ax.legend(loc=2, prop={'size': 12})
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('Changes of Relative Statistical Entropy in the ELV sector per year.png', dpi=360)

#############################################################
#Changes in RSE cumulated for production sector

data_production_cum = data_production.groupby('year').sum()

plt.figure(figsize=(15,6))
ax = plt.gca()
                        
data_production_cum.plot(y='S70', label='S70', color='tab:blue', linewidth=4,  ax=ax)
data_production_cum.plot(y='S100', label='S100', color='tab:red', linewidth=3, ax=ax)
data_production_cum.plot(y='S70-REU', label='S70-REU', color='grey', linewidth=6, ax=ax)
data_production_cum.plot(y='S70-REU-REC', label='S70-REU-REC', color='tab:orange', linewidth=3, ax=ax)

data_production_cum.plot(y='S70D', label='S70D', color='tab:blue', dashes=[3, 0.5], linewidth=4, ax=ax)
data_production_cum.plot(y='S70D-REU', label='S70D-REU', color='grey', dashes=[3, 0.5], linewidth=6, ax=ax)
data_production_cum.plot(y='S70D-REU-REC', label='S70D-REU-REC', color='tab:orange', dashes=[3, 0.5], linewidth=3, ax=ax)
data_production_cum.plot(y='S70D-LFT', label='S70D-LFT', color='tab:brown', dashes=[1, 0.5], linewidth=4, ax=ax)
data_production_cum.plot(y='S70D-REU-REC-LFT', label='S70D-REU-REC-LFT', color='dimgray', dashes=[1, 0.5], linewidth=4, ax=ax)
data_production_cum.plot(y='S100D-REU-REC-LFT', label='S100D-REU-REC-LFT', color='tab:red', dashes=[1, 0.5], linewidth=4, ax=ax)

ax.set_xlabel("Year", fontsize=12)
ax.set_ylabel(r'$\Delta RSE_{cum}$', fontsize=14)
ax.grid(True)
ax.set_ylim([0, 0.3])
ax.legend(loc=2, prop={'size': 12})
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('Changes of Relative Statistical Entropy in the production sector per year.png', dpi=360)

#############################################################
#Changes in RSE cumulated for production and ELV sector

data_total_dH_cum = data_production_cum + data_dH_ELV_cum
data_total_dH_cum

plt.figure(figsize=(15,6))
ax = plt.gca()
                        
data_total_dH_cum.plot(y='S70', label='S70', color='tab:blue', linewidth=4,  ax=ax)
data_total_dH_cum.plot(y='S100', label='S100', color='tab:red', linewidth=4, ax=ax)
data_total_dH_cum.plot(y='S70-REU', label='S70-REU', color='grey', linewidth=4, ax=ax)
data_total_dH_cum.plot(y='S70-REU-REC', label='S70-REU-REC', color='tab:orange', linewidth=4, ax=ax)
data_total_dH_cum.plot(y='S70D', label='S70D', color='tab:blue', dashes=[3, 0.5], linewidth=4, ax=ax)
data_total_dH_cum.plot(y='S70D-REU', label='S70D-REU', color='grey', dashes=[3, 0.5], linewidth=4, ax=ax)
data_total_dH_cum.plot(y='S70D-REU-REC', label='S70D-REU-REC', color='tab:orange', dashes=[3, 0.5], linewidth=4, ax=ax)
data_total_dH_cum.plot(y='S70D-LFT', label='S70D-LFT', color='tab:brown', dashes=[1, 0.5], linewidth=4, ax=ax)
data_total_dH_cum.plot(y='S70D-REU-REC-LFT', label='S70D-REU-REC-LFT', color='dimgray', dashes=[1, 0.5], linewidth=4, ax=ax)
data_total_dH_cum.plot(y='S100D-REU-REC-LFT', label='S100D-REU-REC-LFT', color='tab:red', dashes=[1, 0.5], linewidth=4, ax=ax)

ax.set_xlabel("Year", fontsize=12)
ax.set_ylabel(r'$\Delta RSE_{cum}$', fontsize=14)
ax.set_ylim([0, 0.55])
ax.grid(True)
ax.legend(loc=2, prop={'size': 12})
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('Supplementary information - Changes of Relative Statistical Entropy in the production and ELV sector per year.png', dpi=360)


In [ ]:
#######################################
#Evolution of vehicle inflows per year
input_file = pd.read_csv("_Main_scenario_file.csv", sep=';')  #_Main_scenario_single_car_based_stock_calc
data = input_file

data.loc[:, '1 Inflow S70 total'] = (data.loc[:, '1 Inflow S70 EV'] + data.loc[:, '1 Inflow S70 ICEV'])
data.loc[:, '2 Inflow S100 total'] = (data.loc[:, '2 Inflow S100 EV'] + data.loc[:, '2 Inflow S100 ICEV'])
data.loc[:, '3 Inflow S70D total'] = (data.loc[:, '3 Inflow S70D EV'] + data.loc[:, '3 Inflow S70D ICEV'])
data.loc[:, '4 Inflow S70D-CE+ total'] = (data.loc[:, '4 Inflow S70D-CE+ EV'] + data.loc[:, '4 Inflow S70D-CE+ ICEV'])
data.loc[:, '5 Inflow S100D-CE+ total'] = (data.loc[:, '5 Inflow S100D-CE+ EV'] + data.loc[:, '5 Inflow S100D-CE+ ICEV'])

plt.figure(figsize=(20,7))
ax = plt.gca()

data.plot(y='1 Inflow S70 EV', x='Year', color='tab:blue', label='S70 EV', linewidth=4,  ax=ax)
data.plot(y='1 Inflow S70 ICEV', x='Year', color='tab:blue', label='S70 ICEV', dashes=[4, 0.5], linewidth=3,  ax=ax)
data.plot(y='1 Inflow S70 total', x='Year', color='tab:blue', label='Total inflow S70', linewidth=10,  ax=ax)

data.plot(y='2 Inflow S100 EV', x='Year', color='tab:red', label='S100 EV', linewidth=4, ax=ax)
data.plot(y='2 Inflow S100 ICEV', x='Year', color='tab:red', label='S100 ICEV', dashes=[4, 0.5], linewidth=3, ax=ax)
data.plot(y='2 Inflow S100 total', x='Year', color='tab:red', label='Total inflow S100', linewidth=6,  ax=ax)

data.plot(y='3 Inflow S70D EV', x='Year', color='tab:orange', label='S70D EV', linewidth=4, ax=ax)
data.plot(y='3 Inflow S70D ICEV', x='Year', color='tab:orange', label='S70D ICEV', dashes=[2, 1], linewidth=3, ax=ax)
data.plot(y='3 Inflow S70D total', x='Year', color='tab:orange', label='Total inflow S70D', linewidth=6,  ax=ax)

data.plot(y='4 Inflow S70D-CE+ EV', x='Year', color='tab:gray', label='S70D-LFT EV', linewidth=4, ax=ax)
data.plot(y='4 Inflow S70D-CE+ ICEV', x='Year', color='tab:gray', label='S70D-LFT ICEV', dashes=[3, 1], linewidth=3, ax=ax)
data.plot(y='4 Inflow S70D-CE+ total', x='Year', color='tab:gray', label='Total inflow S70D-LFT', linewidth=10, ax=ax)

data.plot(y='5 Inflow S100D-CE+ EV', x='Year', color='black', label='S100D-LFT EV', linewidth=4, ax=ax)
data.plot(y='5 Inflow S100D-CE+ ICEV', x='Year', color='black', label='S100D-LFT ICEV', dashes=[3, 1], linewidth=3, ax=ax)
data.plot(y='5 Inflow S100D-CE+ total', x='Year', color='black', label='Total inflow S100D-LFT', linewidth=2, ax=ax)

ax.set_ylabel("Inflows of vehicles (produced vehicles, in million)", fontsize=16)
ax.set_xlabel("Year", fontsize=14)
#ax.set_title('Produced vehicles', fontsize=18)
ax.grid(True)
ax.legend(loc=2, prop={'size': 13})
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.savefig('Inflows of Vehicles for different scenarios single car based stock calc.png', dpi=360)


In [ ]:
#######################################
#Outflows from stock scenarios
input_file = pd.read_csv("_Main_scenario_file.csv", sep=';')
data = input_file

data.loc[:, '1 Outflow S70 total'] = (data.loc[:, '1 Outflow S70 EV'] + data.loc[:, '1 Outflow S70 ICEV'])
data.loc[:, '2 Outflow S100 total'] = (data.loc[:, '2 Outflow S100 EV'] + data.loc[:, '2 Outflow S100 ICEV'])
data.loc[:, '3 Outflow S70D total'] = (data.loc[:, '3 Outflow S70D EV'] + data.loc[:, '3 Outflow S70D ICEV'])
data.loc[:, '4 Outflow S70D-CE+ total'] = (data.loc[:, '4 Outflow S70D-CE+ EV'] + data.loc[:, '4 Outflow S70D-CE+ ICEV'])
data.loc[:, '5 Outflow S100D-CE+ total'] = (data.loc[:, '5 Outflow S100D-CE+ EV'] + data.loc[:, '5 Outflow S100D-CE+ ICEV'])

#Evolution of vehicle inflows per year
plt.figure(figsize=(20,7))
ax = plt.gca()

data.plot(y='1 Outflow S70 EV', x='Year', color='tab:blue', label='S70 EV', linewidth=4,  ax=ax)
data.plot(y='1 Outflow S70 ICEV', x='Year', color='tab:blue', label='S70 ICEV', dashes=[4, 0.5], linewidth=3,  ax=ax)
data.plot(y='1 Outflow S70 total', x='Year', color='tab:blue', label='1 Total outflow S70', linewidth=10,  ax=ax)

data.plot(y='2 Outflow S100 EV', x='Year', color='tab:red', label='S100 EV', linewidth=4, ax=ax)
data.plot(y='2 Outflow S100 ICEV', x='Year', color='tab:red', label='S100 ICEV', dashes=[4, 0.5], linewidth=3, ax=ax)
data.plot(y='2 Outflow S100 total', x='Year', color='tab:red', label='2 Total outflow S100', linewidth=6,  ax=ax)

data.plot(y='3 Outflow S70D EV', x='Year', color='tab:orange', label='S70D EV', linewidth=4, ax=ax)
data.plot(y='3 Outflow S70D ICEV', x='Year', color='tab:orange', label='S70D ICEV', dashes=[4, 0.5], linewidth=3, ax=ax)
data.plot(y='3 Outflow S70D total', x='Year', color='tab:orange', label='3 Total outflow S70D', linewidth=6,  ax=ax)

data.plot(y='4 Outflow S70D-CE+ EV', x='Year', color='tab:gray', label='S70D-LFT EV', linewidth=4, ax=ax)
data.plot(y='4 Outflow S70D-CE+ ICEV', x='Year', color='tab:gray', label='S70D-LFT ICEV', dashes=[4, 0.5], linewidth=3, ax=ax)
data.plot(y='4 Outflow S70D-CE+ total', x='Year', color='tab:gray', label='4 Total outflow S70D-LFT', linewidth=10,  ax=ax)

data.plot(y='5 Outflow S100D-CE+ EV', x='Year', color='black', label='S100D-LFT EV', linewidth=4, ax=ax)
data.plot(y='5 Outflow S100D-CE+ ICEV', x='Year', color='black', label='S100D-LFT ICEV', dashes=[4, 0.5], linewidth=3, ax=ax)
data.plot(y='5 Outflow S100D-CE+ total', x='Year', color='black', label='5 Total outflow S100D-LFT', linewidth=2,  ax=ax)

ax.set_ylabel("Outflow of vehicles to ELV treatment (in million)", fontsize=16)
ax.set_xlabel("Year", fontsize=14)
ax.grid(True)
ax.legend(loc=2, prop={'size': 12})
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.savefig('Outflows of Vehicles for different scenarios single car based stock calc.png', dpi=360)
